In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ratings/sentiment.csv


In [2]:
# !nvidia-smi

In [3]:
# import tensorflow as tf
# from transformers import BertTokenizer, TFBertModel

# # Load the pre-trained BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = TFBertModel.from_pretrained('bert-base-uncased')

# # Define a function to generate a sentiment score from a review using BERT
# def generate_sentiment_score(review):
#     # Tokenize the review text
#     inputs = tokenizer.encode_plus(review, add_special_tokens=True, return_tensors='tf')

#     # Pass the tokenized inputs through the BERT model to generate embeddings
#     outputs = model(inputs)
#     embeddings = outputs.last_hidden_state

#     # Compute a sentiment score by averaging the embeddings for each token
#     mean_embedding = tf.reduce_mean(embeddings, axis=1)
#     sentiment_score = tf.keras.layers.Dense(1, activation='sigmoid')(mean_embedding)

#     # Return the sentiment score as a float between 0 and 1
#     return float(sentiment_score.numpy())


In [4]:
# df['sentiment'] = df['review'].apply(lambda x: generate_sentiment_score(x))

In [5]:
df = pd.read_csv('/kaggle/input/ratings/sentiment.csv')
df.head()

,index,book_id,user_id,review,title,rating,sentiment
0,1747,B000N6DDJQ,AUM3YMZ0YRJE0,"WHen I finally started reading this classic, I...",The Scarlet Letter A Romance,5.0,0.451157
1,1807,B000N6DDJQ,AMKZHBOK7VMQR,Hawthorne wrote a masterful work of historical...,The Scarlet Letter A Romance,5.0,0.636971
2,1820,B000N6DDJQ,AWLFVCT9128JV,I don't suppose anything can ruin a book more ...,The Scarlet Letter A Romance,4.0,0.481272
3,1829,B000N6DDJQ,ABN5K7K1TM1QA,"""All have sinned and fall short of the glory o...",The Scarlet Letter A Romance,5.0,0.495467
4,1830,B000N6DDJQ,A3IKBHODOTYYHM,I think the many readers who were forced to pa...,The Scarlet Letter A Romance,4.0,0.480073


In [6]:
# df.to_csv('ratings.csv',index=False)

In [7]:
df.drop(['index'],axis=1,inplace=True)

In [8]:
df = df.loc[~((df.rating.isin([1,2])) & (df['sentiment'] > 0.7))]

In [9]:
df.shape

(17100, 6)

In [10]:
df = df.loc[~((df.rating.isin([4,5])) & (df['sentiment'] < 0.4))]

In [11]:
from surprise import Dataset, Reader
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy

In [12]:
# Combine the rating and sentiment scores into a single score
df['hybrid_score'] = df['rating'] * df['sentiment']

In [13]:
# load data from a file
reader = Reader(line_format='user item rating', sep=',')

In [14]:
data = Dataset.load_from_df(df[['user_id', 'book_id', 'hybrid_score']], reader=reader)

In [15]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=.25)

In [16]:
raw_ratings = data.raw_ratings
raw_ratings[:5]

[('AUM3YMZ0YRJE0', 'B000N6DDJQ', 2.255786806344986, None),
 ('AMKZHBOK7VMQR', 'B000N6DDJQ', 3.1848567724227905, None),
 ('AWLFVCT9128JV', 'B000N6DDJQ', 1.9250879287719724, None),
 ('ABN5K7K1TM1QA', 'B000N6DDJQ', 2.477333843708038, None),
 ('A3IKBHODOTYYHM', 'B000N6DDJQ', 1.9202926158905027, None)]

In [17]:
# Train a KNNWithMeans model on the combined score
model = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
model.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
# Evaluate the model on the test set
from surprise import accuracy
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)

RMSE: 0.5162


In [19]:
from collections import defaultdict

In [20]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [21]:
precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=2)
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.8362089201877935
0.5476681303314147


In [22]:
# Get the user ID for whom you want to make recommendations
user_id = 'AWLFVCT9128JV'

# Get the list of all items (books) in the dataset
items = df['book_id'].unique()

# Predict the rating the user would give to each item and store in a dictionary
item_ratings = {}
for item in items:
    predicted_rating = model.predict(user_id, item).est
    item_ratings[item] = predicted_rating

# Sort the items by predicted rating in descending order and select the top 5
top_items = sorted(item_ratings.items(), key=lambda x: x[1], reverse=True)[:5]

# Print the top 5 recommended books
for i, item in enumerate(top_items):
    print(f"{i+1}. {item[0]}({df[df['book_id']==item[0]].iloc[0].title}) with predicted rating of {item[1]:.2f}")

1. B000PC3Q3G(East of Eden) with predicted rating of 3.62
2. B000PYIUAI(The Autobiography of Benjamin Franklin) with predicted rating of 3.54
3. B000NLWCE8(The Martian Chronicles) with predicted rating of 3.48
4. B00087JQ6Y(Leaves of Grass) with predicted rating of 3.41
5. B0007BNSM4(An inquiry into the nature and causes of the wealth of nations) with predicted rating of 3.40
